In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# 데이터 경로 설정
keypoint_dir = "/home/uk/Desktop/dev_ws/DeepLearning/data/label_data/SYN_word_keypoint/02_sys_word_keypoint.zip/WORD/keypoint/03"
label_dir = "/home/uk/Desktop/dev_ws/DeepLearning/data/label_data/SYN_word_keypoint/02_sys_word_keypoint.zip/WORD/morpheme/03"

# 이미지 크기 및 키포인트 개수 설정
img_size = 128
num_keypoints = 25 * 3  # 예시: 25개의 키포인트, x, y, confidence

# 데이터 로드 및 전처리 함수 (Generator)
def data_generator(keypoint_dir, label_dir, img_size):
    label_map = {}  # 라벨-인덱스 매핑
    label_count = 0

    keypoint_folders = sorted(os.listdir(keypoint_dir))
    label_files = sorted(os.listdir(label_dir))

    for folder, label_file in zip(keypoint_folders, label_files):
        folder_path = os.path.join(keypoint_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        label_path = os.path.join(label_dir, label_file)
        if not os.path.exists(label_path):
            print(f"Error: Label file not found: {label_path}")
            continue

        try:
            with open(label_path, "r") as f:
                label_data = json.load(f)
                label = label_data["data"][0]["attributes"][0]["name"]

            if label not in label_map:
                label_map[label] = label_count
                label_count += 1
            label_idx = label_map[label]

            frame_files = sorted(os.listdir(folder_path))
            for frame_file in frame_files:
                if not frame_file.endswith(".json"):
                    continue

                frame_path = os.path.join(folder_path, frame_file)
                if not os.path.exists(frame_path):
                    print(f"Error: Keypoint file not found: {frame_path}")
                    continue

                try:
                    with open(frame_path, "r") as f:
                        keypoint_data = json.load(f)

                    if "people" in keypoint_data and keypoint_data["people"]:
                        pose_keypoints = np.array(keypoint_data["people"]["pose_keypoints_2d"]).reshape(-1, 3)
                        keypoints = pose_keypoints[:,:2].flatten()

                        img = np.zeros((img_size, img_size, 1), dtype=np.uint8)
                        for i in range(0, len(keypoints), 2):
                            x, y = int(keypoints[i] * img_size / 1920), int(keypoints[i + 1] * img_size / 1080)
                            if 0 <= x < img_size and 0 <= y < img_size:
                                img[y, x] = 255

                        img = img / 255.0  # 정규화
                        yield img, label_idx
                    else:
                        print(f"Error: Keypoint data not found in file: {frame_path}")

                except Exception as e:
                    print(f"Error processing keypoint file: {frame_path}, {e}")
        except Exception as e:
            print(f"Error processing label file: {label_path}, {e}")

# TensorFlow Dataset 생성
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(keypoint_dir, label_dir, img_size),
    output_signature=(
        tf.TensorSpec(shape=(img_size, img_size, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

# 데이터 분할
train_size = int(0.7 * len(list(dataset)))
val_size = int(0.15 * len(list(dataset)))
test_size = len(list(dataset)) - train_size - val_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

# 배치 및 셔플
batch_size = 16
train_dataset = train_dataset.batch(batch_size).shuffle(buffer_size=train_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# CNN 모델 정의
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(img_size, img_size, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(len(set(list(dataset.map(lambda x, y: y).as_numpy_iterator()))), activation="softmax")
])

# 모델 컴파일
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 모델 학습
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# 모델 평가
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")

2025-03-17 19:28:11.273052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742207291.364811  129500 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742207291.393131  129500 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742207291.608106  129500 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742207291.608136  129500 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742207291.608137  129500 computation_placer.cc:177] computation placer alr

: 

In [2]:
!nvidia-smi

Mon Mar 17 19:45:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   46C    P8             12W /   70W |      58MiB /   6144MiB |     26%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----